In [130]:
import numpy as np
import pandas as pd

### Load the Data

In [131]:
fmr_2024 = pd.read_csv("fmr2024.csv")

In [132]:
fmr_2024.head()

,stusps,state,hud_area_code,countyname,metro,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4
0,AL,1,METRO33860M33860,Autauga County,1,100199999,55639,836,913,1092,1383,1753
1,AL,1,METRO19300M19300,Baldwin County,1,100399999,218289,1051,1056,1362,1670,2114
2,AL,1,NCNTY01005N01005,Barbour County,0,100599999,25026,652,656,857,1089,1141
3,AL,1,METRO13820M13820,Bibb County,1,100799999,22374,983,1109,1245,1570,1752
4,AL,1,METRO13820M13820,Blount County,1,100999999,57755,983,1109,1245,1570,1752


In [133]:
print(fmr_2024.columns)

Index(['stusps', 'state', 'hud_area_code', 'countyname', 'metro', 'fips',
       'pop2020', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4'],
      dtype='object')


### Vetting the Data

In [134]:
# Check if any column has missing values
print(fmr_2024.isnull().sum())

stusps           0
state            0
hud_area_code    0
countyname       0
metro            0
fips             0
pop2020          0
fmr_0            0
fmr_1            0
fmr_2            0
fmr_3            0
fmr_4            0
dtype: int64


In [135]:
# Check if any duplicates exist
print(fmr_2024.duplicated().any())
# Check if FIPS is unique
print(fmr_2024.duplicated(subset="fips").sum())
# Make sure rent value is integers
for c in ["fmr_1", "fmr_2", "fmr_3", "fmr_4"]:
    fmr_2024[c] = pd.to_numeric(fmr_2024[c], errors="coerce")
# Check again integers
fmr_2024[["fmr_1", "fmr_2", "fmr_3", "fmr_4"]].apply(lambda x: x.dtype)

False
0


fmr_1    int64
fmr_2    int64
fmr_3    int64
fmr_4    int64
dtype: object

In [136]:
# Glimpse the data
fmr_2024.head()

,stusps,state,hud_area_code,countyname,metro,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4
0,AL,1,METRO33860M33860,Autauga County,1,100199999,55639,836,913,1092,1383,1753
1,AL,1,METRO19300M19300,Baldwin County,1,100399999,218289,1051,1056,1362,1670,2114
2,AL,1,NCNTY01005N01005,Barbour County,0,100599999,25026,652,656,857,1089,1141
3,AL,1,METRO13820M13820,Bibb County,1,100799999,22374,983,1109,1245,1570,1752
4,AL,1,METRO13820M13820,Blount County,1,100999999,57755,983,1109,1245,1570,1752


### Extract columns for later visualization
#### Add a new column 5-digit_fips, for visualization in datawrapper

In [137]:
# get only digits (handles NaN / stray characters)
fips_digits = fmr_2024["fips"].astype(str).str.extract(r"(\d+)")[0].fillna("")
# make a 10-digit string (pad with leading zeros if necessary)
fmr_2024["10digit_fips"] = fips_digits.str.zfill(10)
# take the first 5 digits
fmr_2024["5digit_fips"] = fmr_2024["10digit_fips"].str[:5].str.zfill(5)

In [138]:
# Reorder the column
cols = list(fmr_2024.columns)
cols.remove("5digit_fips")
cols.insert(5, "5digit_fips")
fmr_2024 = fmr_2024[cols]

In [238]:
# Using CA (fips start with zero) to check the 5 digit fips
fmr_2024[fmr_2024["stusps"] == "CA"].head(10)

,stusps,state,hud_area_code,countyname,metro,5digit_fips,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,10digit_fips
187,CA,6,METRO41860MM5775,Alameda County,1,06001,600199999,1661584,1825,2131,2590,3342,3954,0600199999
188,CA,6,NCNTY06003N06003,Alpine County,0,06003,600399999,1159,911,1005,1321,1862,2168,0600399999
189,CA,6,NCNTY06005N06005,Amador County,0,06005,600599999,39023,1128,1135,1347,1898,2286,0600599999
190,CA,6,METRO17020M17020,Butte County,1,06007,600799999,223344,1049,1091,1428,2012,2423,0600799999
191,CA,6,NCNTY06009N06009,Calaveras County,0,06009,600999999,45828,1049,1055,1309,1845,2135,0600999999
192,CA,6,NCNTY06011N06011,Colusa County,0,06011,601199999,21491,823,829,1089,1519,1525,0601199999
193,CA,6,METRO41860MM5775,Contra Costa County,1,06013,601399999,1147788,1825,2131,2590,3342,3954,0601399999
194,CA,6,NCNTY06015N06015,Del Norte County,0,06015,601599999,27692,791,970,1147,1616,1946,0601599999
195,CA,6,METRO40900M40900,El Dorado County,1,06017,601799999,190345,1543,1666,2072,2884,3321,0601799999
196,CA,6,METRO23420M23420,Fresno County,1,06019,601999999,990204,1149,1157,1443,2033,2330,0601999999


In [140]:
# Select the column I need for the visualization
# Each county's Fair Market Rents for each type of units
fmr_2024_1b = fmr_2024[["5digit_fips", "stusps", "countyname", "fmr_1"]]
fmr_2024_2b = fmr_2024[["5digit_fips", "stusps", "countyname", "fmr_2"]]
fmr_2024_3b = fmr_2024[["5digit_fips", "stusps", "countyname", "fmr_3"]]
fmr_2024_4b = fmr_2024[["5digit_fips", "stusps", "countyname", "fmr_4"]]

In [147]:
# Rename the dataframe
fmr_2024_1b = fmr_2024_1b.rename(columns={"fmr_1": "Fair Market Rents_1 bedroom", "stusps": "State Postal Code", "countyname": "County Name"})
fmr_2024_2b = fmr_2024_2b.rename(columns={"fmr_2": "Fair Market Rents_2 bedroom", "stusps": "State Postal Code", "countyname": "County Name"})
fmr_2024_3b = fmr_2024_3b.rename(columns={"fmr_3": "Fair Market Rents_3 bedroom", "stusps": "State Postal Code", "countyname": "County Name"})
fmr_2024_4b = fmr_2024_4b.rename(columns={"fmr_4": "Fair Market Rents_4 bedroom", "stusps": "State Postal Code", "countyname": "County Name"})

### Export the files

In [148]:
fmr_2024.to_csv("fmr_all_counties.csv", index=False)
fmr_2024_1b.to_csv("fmr_1bed.csv", index=False)
fmr_2024_2b.to_csv("fmr_2bed.csv", index=False)
fmr_2024_3b.to_csv("fmr_3bed.csv", index=False)
fmr_2024_4b.to_csv("fmr_4bed.csv", index=False)

### Further Exploration

#### Top

In [155]:
fmr_2024_1b_top10 = fmr_2024_1b.sort_values(by='Fair Market Rents_1 bedroom', ascending=False).head(10)
fmr_2024_1b_top10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_1 bedroom
230,06087,CA,Santa Cruz County,3085
207,06041,CA,Marin County,2818
227,06081,CA,San Mateo County,2818
224,06075,CA,San Francisco County,2818
229,06085,CA,Santa Clara County,2694
228,06083,CA,Santa Barbara County,2651
3134,36087,NY,Rockland County,2451
3133,36085,NY,Richmond County,2451
3131,36081,NY,Queens County,2451
3130,36079,NY,Putnam County,2451


In [157]:
fmr_2024_2b_top10 = fmr_2024_2b.sort_values(by='Fair Market Rents_2 bedroom', ascending=False).head(10)
fmr_2024_2b_top10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_2 bedroom
230,06087,CA,Santa Cruz County,4054
207,06041,CA,Marin County,3359
224,06075,CA,San Francisco County,3359
227,06081,CA,San Mateo County,3359
229,06085,CA,Santa Clara County,3132
228,06083,CA,Santa Barbara County,2994
213,06053,CA,Monterey County,2879
223,06073,CA,San Diego County,2833
1996,25009,MA,Essex County,2827
2070,25017,MA,Middlesex County,2827


In [158]:
fmr_2024_3b_top10 = fmr_2024_3b.sort_values(by='Fair Market Rents_3 bedroom', ascending=False).head(10)
fmr_2024_3b_top10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_3 bedroom
230,06087,CA,Santa Cruz County,5000
224,06075,CA,San Francisco County,4112
227,06081,CA,San Mateo County,4112
207,06041,CA,Marin County,4112
229,06085,CA,Santa Clara County,4011
228,06083,CA,Santa Barbara County,3996
213,06053,CA,Monterey County,3990
223,06073,CA,San Diego County,3819
216,06059,CA,Orange County,3769
214,06055,CA,Napa County,3634


In [159]:
fmr_2024_4b_top10 = fmr_2024_4b.sort_values(by='Fair Market Rents_4 bedroom', ascending=False).head(10)
fmr_2024_4b_top10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_4 bedroom
230,06087,CA,Santa Cruz County,5504
223,06073,CA,San Diego County,4638
228,06083,CA,Santa Barbara County,4528
207,06041,CA,Marin County,4473
227,06081,CA,San Mateo County,4473
224,06075,CA,San Francisco County,4473
216,06059,CA,Orange County,4467
229,06085,CA,Santa Clara County,4425
213,06053,CA,Monterey County,4400
221,06069,CA,San Benito County,4121


#### Bottom

In [209]:
fmr_2024_1b_bot10 = fmr_2024_1b.sort_values(by='Fair Market Rents_1 bedroom', ascending=True).head(10)
fmr_2024_1b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_1 bedroom
4690,72015,PR,Arroyo Municipio,406
4738,72109,PR,Patillas Municipio,406
4712,72057,PR,Guayama Municipio,406
4730,72093,PR,Maricao Municipio,421
4707,72049,PR,Culebra Municipio,421
4745,72123,PR,Salinas Municipio,421
4711,72055,PR,Guánica Municipio,421
4704,72043,PR,Coamo Municipio,421
4757,72147,PR,Vieques Municipio,421
4750,72133,PR,Santa Isabel Municipio,421


In [210]:
fmr_2024_2b_bot10 = fmr_2024_2b.sort_values(by='Fair Market Rents_2 bedroom', ascending=True).head(10)
fmr_2024_2b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_2 bedroom
4757,72147,PR,Vieques Municipio,473
4739,72111,PR,Peñuelas Municipio,473
4750,72133,PR,Santa Isabel Municipio,473
4704,72043,PR,Coamo Municipio,473
4720,72073,PR,Jayuya Municipio,473
4707,72049,PR,Culebra Municipio,473
4745,72123,PR,Salinas Municipio,473
4730,72093,PR,Maricao Municipio,473
4711,72055,PR,Guánica Municipio,473
4713,72059,PR,Guayanilla Municipio,473


In [211]:
fmr_2024_3b_bot10 = fmr_2024_3b.sort_values(by='Fair Market Rents_3 bedroom', ascending=True).head(10)
fmr_2024_3b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_3 bedroom
4713,72059,PR,Guayanilla Municipio,628
4760,72153,PR,Yauco Municipio,628
4711,72055,PR,Guánica Municipio,628
4739,72111,PR,Peñuelas Municipio,628
4688,72011,PR,Añasco Municipio,632
4685,72005,PR,Aguadilla Municipio,632
4684,72003,PR,Aguada Municipio,632
4742,72117,PR,Rincón Municipio,632
4724,72081,PR,Lares Municipio,632
4733,72099,PR,Moca Municipio,632


In [212]:
fmr_2024_4b_bot10 = fmr_2024_4b.sort_values(by='Fair Market Rents_4 bedroom', ascending=True).head(10)
fmr_2024_4b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_4 bedroom
4739,72111,PR,Peñuelas Municipio,630
4711,72055,PR,Guánica Municipio,630
4713,72059,PR,Guayanilla Municipio,630
4760,72153,PR,Yauco Municipio,630
4738,72109,PR,Patillas Municipio,646
4712,72057,PR,Guayama Municipio,646
4690,72015,PR,Arroyo Municipio,646
4719,72071,PR,Isabela Municipio,713
4754,72141,PR,Utuado Municipio,713
4749,72131,PR,San Sebastián Municipio,713


##### A number of the lowest Fair Market Rent values in the dataset are found in counties located in Puerto Rico. 

##### Although Puerto Rico is a territory of the United States, it is neither a state nor located on the U.S. mainland. Therefore, comparisons of rental costs between Puerto Rico and U.S. states may not accurately reflect housing market conditions within the continental United States.

##### For this reason, it is more appropriate to restrict the analysis of lowest-cost counties to those within U.S. states on the mainland, excluding Puerto Rico from direct comparison.

##### [Source: Office of the Historian, U.S. Department of State.](https://history.state.gov/historicaldocuments/frus1952-54v03/d902#:~:text=Puerto%20Rico%20has%20not%20become,territory%20of%20the%20United%20States)

In [214]:
# Compare the rental costs without Puerto Rico's record.
fmr_2024_1b_noPR = fmr_2024_1b[fmr_2024_1b["State Postal Code"] != "PR"]
# Take the bottom 10 among states only
fmr_2024_1b_bot10 = (
    fmr_2024_1b_noPR
    .sort_values(by="Fair Market Rents_1 bedroom", ascending=True)
    .head(10)
)
fmr_2024_1b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_1 bedroom
2606,29221,MO,Washington County,607
2526,29059,MO,Dallas County,607
2527,29061,MO,Daviess County,607
2529,29065,MO,Dent County,607
2530,29067,MO,Douglas County,607
2558,29123,MO,Madison County,607
2557,29121,MO,Macon County,607
2533,29073,MO,Gasconade County,607
2595,29199,MO,Scotland County,607
2554,29115,MO,Linn County,607


In [215]:
fmr_2024_2b_noPR = fmr_2024_2b[fmr_2024_2b["State Postal Code"] != "PR"]
fmr_2024_2b_bot10 = (
    fmr_2024_2b_noPR
    .sort_values(by="Fair Market Rents_2 bedroom", ascending=True)
    .head(10)
)
fmr_2024_2b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_2 bedroom
2538,29083,MO,Henry County,797
2558,29123,MO,Madison County,797
2557,29121,MO,Macon County,797
2556,29119,MO,McDonald County,797
2554,29115,MO,Linn County,797
2552,29111,MO,Lewis County,797
2551,29109,MO,Lawrence County,797
2537,29081,MO,Harrison County,797
2559,29125,MO,Maries County,797
2549,29105,MO,Laclede County,797


In [216]:
fmr_2024_3b_noPR = fmr_2024_3b[fmr_2024_3b["State Postal Code"] != "PR"]
fmr_2024_3b_bot10 = (
    fmr_2024_3b_noPR
    .sort_values(by="Fair Market Rents_3 bedroom", ascending=True)
    .head(10)
)
fmr_2024_3b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_3 bedroom
2530,29067,MO,Douglas County,964
2533,29073,MO,Gasconade County,964
2537,29081,MO,Harrison County,964
2498,29005,MO,Atchison County,964
2606,29221,MO,Washington County,964
2554,29115,MO,Linn County,964
2565,29137,MO,Monroe County,964
2587,29181,MO,Ripley County,964
2589,29185,MO,St. Clair County,964
2594,29197,MO,Schuyler County,964


In [217]:
fmr_2024_4b_noPR = fmr_2024_4b[fmr_2024_4b["State Postal Code"] != "PR"]
fmr_2024_4b_bot10 = (
    fmr_2024_4b_noPR
    .sort_values(by="Fair Market Rents_4 bedroom", ascending=True)
    .head(10)
)
fmr_2024_4b_bot10

,5digit_fips,State Postal Code,County Name,Fair Market Rents_4 bedroom
2538,29083,MO,Henry County,1061
2552,29111,MO,Lewis County,1061
2499,29007,MO,Audrain County,1061
2554,29115,MO,Linn County,1061
2567,29141,MO,Morgan County,1061
2594,29197,MO,Schuyler County,1061
2601,29211,MO,Sullivan County,1061
2537,29081,MO,Harrison County,1070
783,17053,IL,Ford County,1075
782,17051,IL,Fayette County,1075


### Focusing the top and bottom rental cost regions - CA & MO

In [239]:
# Filter rows of counties in California
# View the sub-dataset
fmr_2024[fmr_2024["stusps"] == "CA"].head(10)

,stusps,state,hud_area_code,countyname,metro,5digit_fips,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,10digit_fips
187,CA,6,METRO41860MM5775,Alameda County,1,06001,600199999,1661584,1825,2131,2590,3342,3954,0600199999
188,CA,6,NCNTY06003N06003,Alpine County,0,06003,600399999,1159,911,1005,1321,1862,2168,0600399999
189,CA,6,NCNTY06005N06005,Amador County,0,06005,600599999,39023,1128,1135,1347,1898,2286,0600599999
190,CA,6,METRO17020M17020,Butte County,1,06007,600799999,223344,1049,1091,1428,2012,2423,0600799999
191,CA,6,NCNTY06009N06009,Calaveras County,0,06009,600999999,45828,1049,1055,1309,1845,2135,0600999999
192,CA,6,NCNTY06011N06011,Colusa County,0,06011,601199999,21491,823,829,1089,1519,1525,0601199999
193,CA,6,METRO41860MM5775,Contra Costa County,1,06013,601399999,1147788,1825,2131,2590,3342,3954,0601399999
194,CA,6,NCNTY06015N06015,Del Norte County,0,06015,601599999,27692,791,970,1147,1616,1946,0601599999
195,CA,6,METRO40900M40900,El Dorado County,1,06017,601799999,190345,1543,1666,2072,2884,3321,0601799999
196,CA,6,METRO23420M23420,Fresno County,1,06019,601999999,990204,1149,1157,1443,2033,2330,0601999999


#### Extracted 2-bedroom unit to do a visualization.

In [232]:
# Filter and select the column I need.
fmr_ca_2b = fmr_2024[fmr_2024["stusps"] == "CA"][["5digit_fips", "stusps", "metro", "countyname", "fmr_2"]]
# Check if there are 58 rows (CA has 58 counties)
fmr_ca_2b.info()
fmr_ca_2b = fmr_ca_2b.sort_values(by='fmr_2', ascending=False)
fmr_ca_2b_top10 = fmr_ca_2b.sort_values(by='fmr_2', ascending=False).head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 58 entries, 187 to 244
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   5digit_fips  58 non-null     object
 1   stusps       58 non-null     object
 2   metro        58 non-null     int64 
 3   countyname   58 non-null     object
 4   fmr_2        58 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.7+ KB


In [234]:
# Export the file for visualization
fmr_ca_2b.to_csv("fmr_ca_2b.csv", index=False)
fmr_ca_2b_top10.to_csv("fmr_ca_2b_top10.csv", index=False)

In [218]:
# Filter rows of counties in Missouri
# View the sub-dataset
fmr_2024[fmr_2024["stusps"] == "MO"]

,stusps,state,hud_area_code,countyname,metro,5digit_fips,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,10digit_fips
2496,MO,29,NCNTY29001N29001,Adair County,0,29001,2900199999,25468,542,609,800,1127,1358,2900199999
2497,MO,29,METRO41140M41140,Andrew County,1,29003,2900399999,17554,653,734,964,1223,1331,2900399999
2498,MO,29,NCNTY29005N29005,Atchison County,0,29005,2900599999,5180,601,607,797,964,1096,2900599999
2499,MO,29,NCNTY29007N29007,Audrain County,0,29007,2900799999,25336,601,631,797,1028,1061,2900799999
2500,MO,29,NCNTY29009N29009,Barry County,0,29009,2900999999,35615,603,607,797,1123,1158,2900999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,MO,29,NCNTY29223N29223,Wayne County,0,29223,2922399999,13058,601,699,797,1027,1353,2922399999
2608,MO,29,METRO44180M44180,Webster County,1,29225,2922599999,39127,718,723,921,1274,1434,2922599999
2609,MO,29,NCNTY29227N29227,Worth County,0,29227,2922799999,2001,622,628,825,1163,1259,2922799999
2610,MO,29,NCNTY29229N29229,Wright County,0,29229,2922999999,18256,601,607,797,1120,1210,2922999999


#### Extracted 2-bedroom unit to do a visualization.

In [240]:
# Filter and select the column I need.
fmr_mo_2b = fmr_2024[fmr_2024["stusps"] == "MO"][["5digit_fips", "stusps", "metro", "countyname", "fmr_2"]]
# Check if there are 114 rows (MO has 114 counties)
fmr_mo_2b.info()
# There are 116 entries. cannot find the duplicates for now, will inspect it manually later when doing visualization.
dupe_pairs = (
    fmr_mo_2b
    .duplicated(subset=["5digit_fips", "countyname"], keep=False)
)
fmr_mo_2b = fmr_mo_2b.sort_values(by='fmr_2', ascending=False)
fmr_mo_2b_bot10 = fmr_mo_2b.sort_values(by='fmr_2', ascending=False).head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 2496 to 2611
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   5digit_fips  116 non-null    object
 1   stusps       116 non-null    object
 2   metro        116 non-null    int64 
 3   countyname   116 non-null    object
 4   fmr_2        116 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 5.4+ KB


In [237]:
# Export the file for visualization
fmr_mo_2b.to_csv("fmr_mo_2b.csv", index=False)
fmr_mo_2b_bot10.to_csv("fmr_mo_2b_bot10.csv", index=False)

#### Notes for further exploration in the future

In [241]:
# Metro analysis for different counties, states